<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_animation_icecore_back_800K_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create an animation appearing on the Keeling Curve website
### **Animation: CO<sub>2</sub> record from 800K years ago to now**

**Mauna Loa, Hawaii CO<sub>2</sub> record starting in 1958 and ice-core CO<sub>2</sub> record before 1958**

#### This notebook creates a customizable mp4 animation of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

See an overview of the Keeling Curve notebooks and how to use them at [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb)

----

### [Go to creating the animation section](#steps-to-create-the-animation)

-----

### Sample of the animation produced by this notebook

In [49]:
from base64 import b64encode
from IPython.display import HTML, display
import requests
import pathlib
from ipywidgets import widgets
import functools
import math

def show_sample_animation(anim_out, btn):

    anim_out.clear_output()

    video_url = 'https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_animations/co2_800k.mp4?raw=1'

    r = requests.get(video_url, stream = True)

    download_dir = pathlib.Path('./downloads')
    download_dir.mkdir(exist_ok=True)

    video_name = f'./downloads/co2_800k.mp4'

    with open(video_name,"wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            # writing one chunk at a time 
            if chunk:
                f.write(chunk)
            
    the_animation = open(video_name,'rb').read()

    data_url = "data:video/mp4;base64," + b64encode(the_animation).decode()

    html_str = """
    <video width=800 controls>
            <source src="%s" type="video/mp4">
    </video>
    """ % data_url

    with anim_out:
        display(HTML(html_str))


anim_out = widgets.Output()

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Show sample animation',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)  

button.on_click(functools.partial(show_sample_animation, anim_out))
display(button)

display(anim_out)

Button(button_style='primary', description='Show sample animation', layout=Layout(height='auto', width='auto')…

Output()

## **Notebook Code**

### Import packages

In [50]:
import functools
import math
from time import perf_counter
import requests
import re
import pandas as pd
import numpy as np
from scipy import interpolate
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker
from matplotlib import rc,rcParams
from matplotlib.path import Path
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.transforms
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import urllib
import os

%matplotlib inline

In [51]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [52]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

### Set directories and file names

In [53]:
# Get MLO data from the keelingcurve_notebooks github repository
mlo_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_full_record_now_span.csv'

# Get the icecore dataset going back 2K years to 154 CE
# This is a subset of the full dataset found at https://doi.org/10.25919/5bfe29ff807fb
# The data subset is CO2 data from ice cores up to and including 1957
icecore_2K_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/icecore_data/Law_Dome_GHG_2000years_co2_by_age_wo_firn_to_1957.csv'

# Get the icecore dataset back 800K years located at 
# the site: https://www.ncei.noaa.gov/access/paleo-search/study/6091
# the file: https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc-co2-2008.txt
# with a reference to the journal articls at https://doi.org/10.1038/nature06949
icecore_800K_url = 'https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc-co2-2008.txt'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

animation_base_name = 'co2_800k'

animation_dir = pathlib.Path("./animations")
animation_dir.mkdir(exist_ok=True)

animation_file = f'{animation_base_name}.mp4'

## Functions to convert dates into various formats

In [54]:
# Function to convert datetime to a float

# source, https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

In [55]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

## **Load in data and prepare it for plotting**

### **Load in MLO record**

In [56]:
df_mlo = pd.read_csv(mlo_data_file,sep=',',comment='"')

### Get MLO seasonally adjusted data

In [57]:
df_mlo = df_mlo[['date_seas_adj', 'co2_seas_adj']].copy()

#### Remove rows with fill values NaN

In [58]:
df_mlo = df_mlo.dropna()

#### Convert to numpy arrays for plotting

Will use this to annotate the plot at various years

In [59]:
mlo_date = df_mlo['date_seas_adj'].to_numpy()
mlo_co2 = df_mlo['co2_seas_adj'].to_numpy()

### Filter MLO seasonally adjusted data to first value of each year

This is for the animation to be one frame per year since it is on a large time scale when the ice-core data is combined with it

In [60]:
# First get the floor of each date and then get unique values
# Apply floor function to each element in the column 'date_seas_adj'
df_mlo['start_year'] = df_mlo['date_seas_adj'].apply(lambda x: math.floor(x))

last_year = df_mlo['date_seas_adj'].iloc[-1]
last_co2 = df_mlo['co2_seas_adj'].iloc[-1]

# Drop duplicates except for the first value (which will be the first value of each year)
df_mlo_start_years = df_mlo.drop_duplicates('start_year').copy()

df_mlo_start_years.reset_index(inplace=True, drop=True)

# Remove the start_year column since it was only used for getting unique values
df_mlo_start_years.drop('start_year', axis=1, inplace=True)

# Add the last values of CO2
df = {'date_seas_adj': last_year, 'co2_seas_adj': last_co2}
df_mlo_start_years = df_mlo_start_years.append(df, ignore_index = True)

# Add back in the last date and value

print(df_mlo_start_years.head())

print(df_mlo_start_years.tail())

   date_seas_adj  co2_seas_adj
0    1958.202740        314.43
1    1959.041096        315.52
2    1960.040984        316.38
3    1961.041096        316.84
4    1962.041096        317.88
    date_seas_adj  co2_seas_adj
62    2020.040984        413.24
63    2021.041096        415.08
64    2022.041096        417.95
65    2023.041096        419.17
66    2023.126027        419.55


#### Convert to numpy arrays for plotting

Will use this data in the animation

In [61]:
mlo_yearly_date = df_mlo_start_years['date_seas_adj'].to_numpy()
mlo_yearly_co2 = df_mlo_start_years['co2_seas_adj'].to_numpy()

### **Load in the icecore record back 2K years**

In [62]:
df_icecore_2K = pd.read_csv(icecore_2K_data_file, sep=',', comment='#', skipinitialspace=True)

df_icecore_2K.head()

,date_ce,co2
0,154.0,278.2
1,175.0,277.7
2,241.0,280.4
3,266.0,278.4
4,274.0,280.2


### **Load in the icecore record back 800K years**
The date column is in BP units which stands for "before present year"


In [63]:
response = requests.get(icecore_800K_url)
file_text = response.text
text_lines = file_text.split('\n')

#### Use data from section 3 of ice-core file edc-co2-2008.txt labeled: "Composite CO2 record"

**Section to start with**

3. Composite CO2 record (0-800 kyr BP)

**Section to stop with**

end of file

In [64]:
start_section = [i for i in range(len(text_lines))
               if text_lines[i].startswith('3. Composite CO2 record (0-800 kyr BP)')][0]

# Select to end of file since no more sections after Section 3
section_lines = text_lines[start_section:]

# start data selection after line "Age(yrBP)    CO2(ppmv)"
header_end = [i for i in range(len(section_lines))
              if section_lines[i].startswith('Age(yrBP)')][0]

start_data = header_end + 1

data_lines = section_lines[start_data:]

data_lines[1:10]

['268           274.9\r',
 '279           277.9\r',
 '395           279.1\r',
 '404           281.9\r',
 '485           277.7\r',
 '559           281.1\r',
 '672           282.2\r',
 '754           280.1\r',
 '877           278.4\r']

Remove trailing return character

In [65]:
data_list = [line.rstrip() for line in data_lines]
data_list[0:10]

['137           280.4',
 '268           274.9',
 '279           277.9',
 '395           279.1',
 '404           281.9',
 '485           277.7',
 '559           281.1',
 '672           282.2',
 '754           280.1',
 '877           278.4']

Read data list into a Pandas dataframe and split into columns date_ce and co2

In [66]:
df_icecore_800K = pd.DataFrame(data_list)
df_icecore_800K.columns = ['data']

df_icecore_800K[['date_bp', 'co2']] = df_icecore_800K['data'].str.split(" ", 1, expand=True)
df_icecore_800K.drop('data', axis=1, inplace=True)

df_icecore_800K.head()

,date_bp,co2
0,137,280.4
1,268,274.9
2,279,277.9
3,395,279.1
4,404,281.9


Remove empty rows and convert columns from strings to numbers

In [67]:
# First change empty strings to NaN. Then can easily drop NaN rows
df_icecore_800K['date_bp'].replace('', np.nan, inplace=True)

df_icecore_800K = df_icecore_800K.dropna()

df_icecore_800K = df_icecore_800K.astype(float)

### Combine 800K back and 2K back together
Exclude 800K dataset points that overlap with the start of the 2K dataset

The date in the 800K dataset is in BP years which means (before present) years. Here in the dataset, present year is 1950

First convert 800K dataset from dates BP (before present) to CE

years before present = 1950 - year ce

year ce = 1950 - years before present

In [68]:
df_icecore_800K['date_ce'] = 1950 - df_icecore_800K['date_bp']
df_icecore_800K[0:10]

,date_bp,co2,date_ce
0,137.0,280.4,1813.0
1,268.0,274.9,1682.0
2,279.0,277.9,1671.0
3,395.0,279.1,1555.0
4,404.0,281.9,1546.0
5,485.0,277.7,1465.0
6,559.0,281.1,1391.0
7,672.0,282.2,1278.0
8,754.0,280.1,1196.0
9,877.0,278.4,1073.0


Sort date ascending

In [69]:
df_icecore_800K = df_icecore_800K.sort_values(by=['date_ce'], ascending=True)

# And reset the index
df_icecore_800K = df_icecore_800K.reset_index(drop=True)

df_icecore_800K.head()

,date_bp,co2,date_ce
0,798512.0,191.0,-796562.0
1,797099.0,188.4,-795149.0
2,796467.0,189.3,-794517.0
3,795202.0,195.2,-793252.0
4,794608.0,199.4,-792658.0


Remove data points from the 800K dataset that overlap with the 2K dataset

In [70]:
min_2K = min(df_icecore_2K['date_ce'])

df_icecore_800K = df_icecore_800K[df_icecore_800K['date_ce'] < min_2K]

Remove 'date_bp' column to have matching columns when concatenate 2K and 800K datasets

In [71]:
df_icecore_800K.drop('date_bp', axis=1, inplace=True)

# move date_ce column before co2
df_icecore_800K = df_icecore_800K.reindex(columns=['date_ce', 'co2'])

df_icecore_800K.head()

,date_ce,co2
0,-796562.0,191.0
1,-795149.0,188.4
2,-794517.0,189.3
3,-793252.0,195.2
4,-792658.0,199.4


Concatenate the icecore 800K and 2K datasets

In [72]:
df_combined_icecore = pd.concat([df_icecore_800K, df_icecore_2K], ignore_index=True)

### Combine icecore data with MLO data

Rename columns so will have same column names when combine with the icecore dataframe

In [73]:
df_mlo_start_years.columns = ['date_ce', 'co2']

df_mlo_start_years.head()

,date_ce,co2
0,1958.202740,314.43
1,1959.041096,315.52
2,1960.040984,316.38
3,1961.041096,316.84
4,1962.041096,317.88


Concatenate combined icecore with MLO data

and sort ascending

In [74]:
df_combined = pd.concat([df_combined_icecore, df_mlo_start_years], ignore_index=True)

# And reset the index
df_combined = df_combined.reset_index(drop=True)

# sort ascending
df_combined = df_combined.sort_values(by=['date_ce'], ascending=True)

df_combined[0:10]

df_combined[:-10]

,date_ce,co2
0,-796562.000000,191.00
1,-795149.000000,188.40
2,-794517.000000,189.30
3,-793252.000000,195.20
4,-792658.000000,199.40
...,...,...
1396,2010.041096,388.49
1397,2011.041096,391.26
1398,2012.040984,393.07
1399,2013.041096,395.63


Filter the values to only include one value per year

In [75]:
# First get the floor of each date and then get unique values
# Apply floor function to each element in the column 'date_seas_adj'
df_combined['start_year'] = df_combined['date_ce'].apply(lambda x: math.floor(x))

# Drop duplicates except for the first value (which will be the first value of each year)
df_combined_start_years = df_combined.drop_duplicates('start_year').copy()

df_combined_start_years.reset_index(inplace=True, drop=True)

# Remove the start_year column since it was only used for getting unique values
df_combined_start_years.drop('start_year', axis=1, inplace=True)

df_combined = df_combined_start_years.copy()

print(df_combined.head())
print(df_combined.tail())


    date_ce    co2
0 -796562.0  191.0
1 -795149.0  188.4
2 -794517.0  189.3
3 -793252.0  195.2
4 -792658.0  199.4
          date_ce     co2
1309  2019.041096  410.79
1310  2020.040984  413.24
1311  2021.041096  415.08
1312  2022.041096  417.95
1313  2023.041096  419.17


### Convert back into units BP (before present)

Today is the present date.

Run function to get todays date in decimal year format


In [76]:
todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

Convert combined data from date_ce to date_bp (before present now)

In [77]:
# years before now = present year - date ce
df_combined['date_bp'] = todays_decimal - df_combined['date_ce']

And sort the data ascending

In [78]:
df_combined = df_combined.sort_values(by=['date_bp'], ascending=True)

#### Convert the dataframe into numpy arrays for plotting

In [79]:
combined_years_before_now = df_combined['date_bp'].to_numpy()

combined_co2 = df_combined['co2'].to_numpy()

## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

In [80]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)

In [81]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

## **Define functions that can modify the animated graphic**

### Function to set fonts and linewidth properties

In [82]:
def set_matplotlib_properties():
    
    # Set default properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This causes matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set axes properties of the graphic

In [83]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)

    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=7, width=1)
    
    tick_spacing = 50
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

    ax.tick_params(which='minor', direction='in', length=4)
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold')

    ax.set_xlabel(xlabel, fontweight='bold', fontsize=12, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    # Set axes limits last 
    # If did before, setting xtick labels past xmin & xmax would have
    # extended the plot limits
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to create x-axis text tick labels

In [84]:
def create_xtick_labels(ax, xmin, xmax):
    
    xrange = range(xmin, xmax)
    
    date_labels = list(range(0, 1000, 100))
    date_tick_labels = [str(x) for x in date_labels]
    
    date_tick_positions = list(range(0, -1000000, -100000))

    plt.xticks(date_tick_positions, date_tick_labels)

    ax.tick_params(axis='x', pad=7)

### Function to place the titles

In [85]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [86]:
def add_inset_label(ax, label_text):
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top')


### Function to add arrow annotations

In [87]:
def get_arrow_annotations_placement(ax, todays_decimal, mlo_date, mlo_co2):
    
    mlo_date_before_now = todays_decimal - mlo_date

    #  1960, 1980, 2000, 2020
    # In years before now
    x_1960 = todays_decimal - 1960
    x_1980 = todays_decimal - 1980
    x_2000 = todays_decimal - 2000
    x_2020 = todays_decimal - 2020

    # Get a spline fit of the seasonal CO2 data
    spl_fit = interpolate.splrep(-mlo_date_before_now, mlo_co2)

    # Evaluate at negative years before now value since this is
    # how the data is plotted with x labels replacing xaxis values later
    spl_val_1960 = interpolate.splev(-x_1960, spl_fit)
    spl_val_1980 = interpolate.splev(-x_1980, spl_fit)
    spl_val_2000 = interpolate.splev(-x_2000, spl_fit)
    spl_val_2020 = interpolate.splev(-x_2020, spl_fit)

    spl_values = {}
    spl_values['1960'] = spl_val_1960
    spl_values['1980'] = spl_val_1980
    spl_values['2000'] = spl_val_2000
    spl_values['2020'] = spl_val_2020

    
    # Annotation start
    x_start = -1000

    return x_start, spl_values

### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [88]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        self.time = perf_counter()

        return self

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))

            self.time = perf_counter() - self.time
            self.readout = f'Time to create animation: {self.time:.3f} seconds'
            print(self.readout)



<a name='steps-to-create-the-animation'></a>
# **Steps to create the animation**

### Comment out any function in the overall function ['create_animation'](#function-to-create-the-animation) to remove an element from the final animation

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [89]:
set_matplotlib_properties()

## **Set plot limits and labeling**

#### Get todays date in various formats for labeling

In [90]:
todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

### Set titles and axes labels

In [91]:
xlabel = 'Thousands of Years Ago'
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ''
title2 = 'Mauna Loa Data starting in 1958. Ice-core data before 1958.'

### Set yaxis min and max limits

In [92]:
ymin = 150
ymax = 430

### Set xaxis min and max limits

In [93]:
xmin = -825000
xmax = 25000

<a name="function-to-create-the-animation"></a>
## **Function to create and save the animation**

In [94]:
def create_animation(msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years_before_now, combined_co2, mlo_date, mlo_co2, fps, dpi, btn):

    msg_out.clear_output()
    anim_out.clear_output()

    # create a figure container
    # and set aspect ration of the animation (width, height)
    fig = plt.figure(figsize=(10,6))

    ax =fig.gca()

    fig_width, fig_height = fig.get_size_inches()

    # set width in inches
    width_in = 10
    height_in = (fig_height / fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    fig.subplots_adjust(left=0.11, bottom=0.1, right=0.96, top=0.56, wspace=None, hspace=None)


    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Animation') as progress:

        # -------------
        # Plot the data
        # -------------

        # -------------------------------------------
        # Create the plot but don't display it
        #
        # The variable 'line_of_plot' is used 
        # to display pieces of the plot at each frame
        # -------------------------------------------

        # Plot years back in time (left of 0) so use negative of values
        # Because want increasing years before now to appear farther left on x axis
  
        # to animate left to right, flip the arrays
        x = combined_years_before_now[::-1]
        y = combined_co2[::-1]

        line_of_plot, = ax.plot(-x, y, '-', color='black', lw=2)
        
        # -----------------------------------------------------
        # Change xaxis to use date labels and not decimal dates
        # -----------------------------------------------------
        create_xtick_labels(ax, xmin, xmax)

        # ---------------
        # Add date arrows
        # ---------------

        x_start, spl_values = get_arrow_annotations_placement(ax, todays_decimal, mlo_date, mlo_co2)

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        inset_text = f'Last updated {todays_date_modyyr}'
        add_inset_label(ax, inset_text)

        # ----------------------------------------
        # Set plot properties

        # Set after the other figure properties to
        # update the default properties
        # -----------------------------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)

        # ====================
        # Create the animation
        # ====================

        def apply_annotation(x, frame_number):

            annotations = []

            if x[frame_number] > 3000:

                return annotations

            for anot_year, co2_value in spl_values.items():

                anot_year_int = int(anot_year)

                mlo_before_now = anot_year_int - todays_decimal

                near_year = math.isclose(-x[frame_number], mlo_before_now, abs_tol = 0.5)

                if near_year:

                    arrow_annotation = ax.annotate(anot_year,
                            xy=[x_start, co2_value],
                            xytext=[-60, 0], 
                            verticalalignment = "center",
                            arrowprops=dict(arrowstyle='->',
                                            relpos=(0, 0.5), lw=1),           
                            fontsize=11,
                            textcoords="offset points")

                    annotations.append(arrow_annotation)

            return annotations


        annotations = []
        def init():
            return line_of_plot, annotations

        # -----------------------------
        # function to update each frame
        # -----------------------------

        def update(frame_number):

            line_of_plot.set_data(-x[:frame_number], y[:frame_number])

            annotations = apply_annotation(x, frame_number)

            return line_of_plot, annotations

        number_of_frames = len(combined_years_before_now)

        anim = animation.FuncAnimation(fig, func=update, init_func=init, frames = number_of_frames, interval=1, blit=False, save_count=number_of_frames+100)

        #When testing, use a samaller set of frames
        #anim = animation.FuncAnimation(fig, func=update, frames = 200, interval=5, blit=False)

        # Save the animation
        anim.save(f'{animation_dir}/{animation_file}', fps=fps, dpi=dpi)

        # display the animation in the output cell
        the_animation = open(f'{animation_dir}/{animation_file}','rb').read()

        with anim_out:

            data_url = "data:video/mp4;base64," + b64encode(the_animation).decode()
            
            html_str = """
            <video width=800 controls>
                    <source src="%s" type="video/mp4">
            </video>
            """ % data_url
            
            display(HTML(html_str))

<a name='button-to-create-animation'></a>
## **Button to create and save the animation**

**The animation takes about 1 1/2 minutes to create at fps=60 and dpi=72**

In [95]:
msg_out = widgets.Output()
anim_out = widgets.Output()

# frames per secod
fps = 60

# dpi of animation
dpi = 72

button = widgets.Button(description='Create animation', button_style='primary')
button.on_click(functools.partial(create_animation, msg_out, anim_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years_before_now, combined_co2, mlo_date, mlo_co2, fps, dpi))
display(button)

display(msg_out)
display(anim_out)

# If the animation is not created, there is most likely an error in the animation functions used
# to create the plot. Turn on console to diagnose errors.


Button(button_style='primary', description='Create animation', style=ButtonStyle())

Output()

Output()

## **Button to download the animation file**

In [96]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(animation_file)
    except:
        with msg_out:
            print("Need to create the animation first.")


msg_out = widgets.Output()

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Animation File',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Animation File', layout=Layout(height='auto', width='auto…

Output()